Here, the data is loaded into a pandas dataframe from my Google drive. The data is posted with the paper by Hou et al., at https://translational-medicine.biomedcentral.com/articles/10.1186/s12967-020-02620-5#availability-of-data-and-materials

In [2]:
import pandas as pd

sepsis_df = pd.read_csv("/content/drive/MyDrive/CS 598 - Deep Learning for Healthcare/Final Project/Reproducing Septic Paper/sepsis_data.csv")

Here, we can see the columns from this dataset, which include demographic information, vital signs, and lab results.

In [3]:
for col in sepsis_df.columns:
  print(col)

icustay_id
hadm_id
intime
outtime
dbsource
suspected_infection_time_poe
suspected_infection_time_poe_days
specimen_poe
positiveculture_poe
antibiotic_time_poe
blood_culture_time
blood_culture_positive
age
gender
is_male
ethnicity
race_white
race_black
race_hispanic
race_other
metastatic_cancer
diabetes
first_service
hospital_expire_flag
thirtyday_expire_flag
icu_los
hosp_los
sepsis_angus
sepsis_martin
sepsis_explicit
septic_shock_explicit
severe_sepsis_explicit
sepsis_nqf
sepsis_cdc
sepsis_cdc_simple
elixhauser_hospital
vent
sofa
lods
sirs
qsofa
qsofa_sysbp_score
qsofa_gcs_score
qsofa_resprate_score
aniongap_min
aniongap_max
bicarbonate_min
bicarbonate_max
creatinine_min
creatinine_max
chloride_min
chloride_max
glucose_min
glucose_max
hematocrit_min
hematocrit_max
hemoglobin_min
hemoglobin_max
lactate_min
lactate_max
lactate_mean
platelet_min
platelet_max
potassium_min
potassium_max
inr_min
inr_max
sodium_min
sodium_max
bun_min
bun_max
bun_mean
wbc_min
wbc_max
wbc_mean
heartrate_min
he

We confirm that there are no missing values in the target, which is thirtyday_expire_flag, indicating whether or not the patient died within 30 days.

In [4]:
sepsis_df["thirtyday_expire_flag"].isna().sum()

0

We look at the value counts for the target, seeing that there are 3670 patients who survived, and 889 patients who died within 30 days. We see that we have a class imbalance, with 80.5% of patients surviving.

In [5]:
sepsis_df["thirtyday_expire_flag"].value_counts()

0    3670
1     889
Name: thirtyday_expire_flag, dtype: int64

Now, we select the target.

In [6]:
y = sepsis_df["thirtyday_expire_flag"]

Next, we drop columns that are not used for prediction. Note that gender and ethnicity are one-hot encoded, and these one-hot encoded variables are still included in the dataset.

In [7]:
to_drop = ["icustay_id",
           "hadm_id",
           "intime",
           "outtime",
           "dbsource",
           "suspected_infection_time_poe",
           "suspected_infection_time_poe_days",
           "specimen_poe",
           "positiveculture_poe",
           "antibiotic_time_poe",
           "blood_culture_time",
           "blood_culture_positive",
           "gender",
           "ethnicity",
           "hospital_expire_flag",
           "thirtyday_expire_flag",
           "first_service"]

X = sepsis_df.drop(columns = to_drop)

We are now left with only numerical features. We perform mean value imputation on all columns to fill in missing values.

In [8]:
X = X.apply(lambda x:x.fillna(x.mean()), axis = 0).values

Now, we ust the StandardScaler from SciKit-Learn to normalize the data so that each feature has mean 0 and standard deviation 1.

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

We will use a memory profile to check the computational requirements.

In [10]:
!pip install memory_profiler
%load_ext memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=5ac90980dbf2409b69a3906c93551b69ca54ebd75e8927eebe8ffa763c018e6c
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


Next, an autoencoder is used for dimensionality reduction and feature construction, reducing the number of dimensions to 40. We begin by convert the data to tensors, and making a data loader.

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64

X = torch.Tensor(X)
y = torch.LongTensor(y.values)

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size = batch_size)

Here, we define the autoencoder.

In [12]:
# Define model
class AE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Linear(89, 40)
        self.decoder = nn.Linear(40, 89)

    def forward(self, x):
        #print(x.shape)
        x = F.relu(self.encoder(x))
        x = F.sigmoid(self.decoder(x))
        return x


model = AE()

To train the autoencoder, we use MSELoss and an Adam optimizer with learning rate 0.001.

In [13]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

We define our training function. Note that we ignore `y` for training the autoencoder, as we are trying to compress the features.

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute reconstruction error
        pred = model(X)
        loss = loss_fn(pred, X)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Now, we train the autoencoder.

In [16]:
%%time
%%memit

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.843112  [    0/ 4559]
Epoch 2
-------------------------------
loss: 0.842898  [    0/ 4559]
Epoch 3
-------------------------------
loss: 0.842697  [    0/ 4559]
Epoch 4
-------------------------------
loss: 0.842529  [    0/ 4559]
Epoch 5
-------------------------------
loss: 0.842338  [    0/ 4559]
Epoch 6
-------------------------------
loss: 0.842187  [    0/ 4559]
Epoch 7
-------------------------------
loss: 0.842009  [    0/ 4559]
Epoch 8
-------------------------------
loss: 0.841850  [    0/ 4559]
Epoch 9
-------------------------------
loss: 0.841690  [    0/ 4559]
Epoch 10
-------------------------------
loss: 0.841551  [    0/ 4559]
Epoch 11
-------------------------------
loss: 0.841408  [    0/ 4559]
Epoch 12
-------------------------------
loss: 0.841274  [    0/ 4559]
Epoch 13
-------------------------------
loss: 0.841135  [    0/ 4559]
Epoch 14
-------------------------------
loss: 0.841013  [    0/ 4559]
Epoch 15
------

Now that we've trained the autoencoder, we use the encoder alone to transform the features.

In [17]:
enc_X = F.relu(model.encoder(X)).detach().numpy()

We split the data into a training set and a testing set, which we will use to train and evaluate models. The training set is 80% of the data, while the testing set is 20% of the data.

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(enc_X, y, test_size = .2, stratify = y, random_state = 0)

Next, we train our first model, which is logistic regression. We use the model classifier from SciKit-Learn, and fit it to the training data. We evaluate the performance by computing the accuracy, precision, recall, and AUC on the testing data.

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

clf = LogisticRegression(random_state = 0, max_iter = 10000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8651315789473685
Precision: 0.7570093457943925
Recall: 0.4550561797752809
AUC: 0.7098169182255151


Our next model is a support vector machine classifier. Again, we use the model from SciKit-Learn. Below, I include the SVC with the default kernel: rbf. I experimented with other kernels, and found that this gave the best performance.

In [20]:
from sklearn.svm import SVC

clf = SVC(random_state = 0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8662280701754386
Precision: 0.9
Recall: 0.3539325842696629
AUC: 0.6721978997642591


Next, I used a random forest classifier.

In [21]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state = 0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8618421052631579
Precision: 0.9333333333333333
Recall: 0.3146067415730337
AUC: 0.6545785751461899


Now, we try a gradient boosting classifier. Note that the result should be essentially the same as XGBoost, except that XGBoost typically runs faster.

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state = 0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8607456140350878
Precision: 0.8311688311688312
Recall: 0.3595505617977528
AUC: 0.6709196950678138


Next, we try a $k$-nearest neighbors classifier. From experimenting with the value our $k$, we found that $k = 1$ gave the best performance (though this is most likely overfitting).

In [23]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8081140350877193
Precision: 0.5114503816793893
Recall: 0.37640449438202245
AUC: 0.6446055169457796
